# Feature Selection based on SelectKBest and RFE

Приведено сравнение двух способов отбора признаков:
* с помощью модели [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html) из библиотеки scikit-learn;
* с помощью метода рекурсивного исключения признаков (recursive feature elimination, [RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html?highlight=rfe#sklearn.feature_selection.RFE)).

Для сравнения используется модель  LGBMClassifier.

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import mannwhitneyu

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score

import lightgbm as lgbm

### Описание датасета

**Home Ownership** - домовладение  
**Annual Income** - годовой доход  
**Years in current job** - количество лет на текущем месте работы  
**Tax Liens** - налоговые льготы  
**Number of Open Accounts** - количество открытых счетов  
**Years of Credit History** - количество лет кредитной истории  
**Maximum Open Credit** - наибольший открытый кредит  
**Number of Credit Problems** - количество проблем с кредитом  
**Months since last delinquent** - количество месяцев с последней просрочки платежа  
**Bankruptcies** - банкротства  
**Purpose** - цель кредита  
**Term** - срок кредита  
**Current Loan Amount** - текущая сумма кредита  
**Current Credit Balance** - текущий кредитный баланс  
**Monthly Debt** - ежемесячный долг  
**Credit Score** - кредитный рейтинг   
**Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

### Загрузка данных

In [2]:
train = pd.read_csv('course_project_train.csv')
train.shape

(7500, 17)

In [3]:
# целевая переменная
target = 'Credit Default'
# числовые признаки
num_features = list(train.select_dtypes(exclude='object').columns)
num_features.remove(target)
# категориальные признаки
cat_features = list(train.select_dtypes(include='object').columns)

### Создание новых признаков 

Чтобы было из чего выбирать, создадим новые признаки путем вычисления разности между некоторыми числовыми признаками.

In [4]:
def new_features(data):
    df = pd.DataFrame()
    feature_list = ['Annual Income', 'Maximum Open Credit', 
                    'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt'
                   ]
    for col1 in feature_list:
        for col2 in feature_list:
            if col1 != col2:
                df[col1 + '-' + col2 + '_' + 'diff'] = data[col1] - data[col2]
                #df[col1 + '-' + col2 + '_' + 'ratio'] = data[col1] / data[col2]
    return df

In [5]:
features = new_features(train)
features.head()

,Annual Income-Maximum Open Credit_diff,Annual Income-Current Loan Amount_diff,Annual Income-Current Credit Balance_diff,Annual Income-Monthly Debt_diff,Maximum Open Credit-Annual Income_diff,Maximum Open Credit-Current Loan Amount_diff,Maximum Open Credit-Current Credit Balance_diff,Maximum Open Credit-Monthly Debt_diff,Current Loan Amount-Annual Income_diff,Current Loan Amount-Maximum Open Credit_diff,Current Loan Amount-Current Credit Balance_diff,Current Loan Amount-Monthly Debt_diff,Current Credit Balance-Annual Income_diff,Current Credit Balance-Maximum Open Credit_diff,Current Credit Balance-Current Loan Amount_diff,Current Credit Balance-Monthly Debt_diff,Monthly Debt-Annual Income_diff,Monthly Debt-Maximum Open Credit_diff,Monthly Debt-Current Loan Amount_diff,Monthly Debt-Current Credit Balance_diff
0,-203873.0,-99517912.0,434701.0,474173.0,203873.0,-99314039.0,638574.0,678046.0,99517912.0,99314039.0,99952613.0,99992085.0,-434701.0,-638574.0,-99952613.0,39472.0,-474173.0,-678046.0,-99992085.0,-39472.0
1,-156243.0,760519.0,630515.0,1007114.0,156243.0,916762.0,786758.0,1163357.0,-760519.0,-916762.0,-130004.0,246595.0,-630515.0,-786758.0,130004.0,376599.0,-1007114.0,-1163357.0,-246595.0,-376599.0
2,-431022.0,-99248587.0,443023.0,737761.0,431022.0,-98817565.0,874045.0,1168783.0,99248587.0,98817565.0,99691610.0,99986348.0,-443023.0,-874045.0,-99691610.0,294738.0,-737761.0,-1168783.0,-99986348.0,-294738.0
3,657668.0,683672.0,709213.0,793730.0,-657668.0,26004.0,51545.0,136062.0,-683672.0,-26004.0,25541.0,110058.0,-709213.0,-51545.0,-25541.0,84517.0,-793730.0,-136062.0,-110058.0,-84517.0
4,390428.0,650424.0,682955.0,769084.0,-390428.0,259996.0,292527.0,378656.0,-650424.0,-259996.0,32531.0,118660.0,-682955.0,-292527.0,-32531.0,86129.0,-769084.0,-378656.0,-118660.0,-86129.0


Получили новые признаки. Объединим их с нашим датасетом.

In [6]:
train = pd.concat([train, features], axis=1)
train.shape

(7500, 37)

Переопределим список числовых признаков `num_features`.

In [7]:
num_features = list(train.select_dtypes(exclude='object').columns)
num_features.remove(target)
len(num_features)

32

## 1. Отбор признаков с помощью модели SelectKBest

Подготовим данные для модели. Отберем только числовые признаки и заменим в них пропуски нулями.

In [8]:
X = train.drop(target, axis=1)
X = X[num_features].fillna(0)
y = train[target]

Модель SelectKBest использует для отбора признаков p-value, полученные по расчетам F-статистики однофакторного дисперсионного анализа (ANOVA). ANOVA применяют для сравнения средних значений двух или более *нормально* распределенных групп. Но в документации SelectKBest про проверку на нормальность ничего не сказано 🤔, что вызывает подозрение.
Закрыв глаза на требование нормальности, попробуем отобрать 10 лучших признаков.

In [9]:
fs = SelectKBest(score_func=f_classif, k=10) 
X_selected = fs.fit_transform(X, y) # массив значений отобранных признаков

ind = list(fs.get_support(True)) # индексы отобранных признаков
selected_features_kBest = [num_features[i] for i in ind] # список отобранных признаков
# готовим таблицу с результатами
is_chosen = np.where(fs.get_support(), '✅', '❌')
results = pd.DataFrame({'feature': num_features, 'pvalue': fs.pvalues_, 'is_chosen': is_chosen})
results = results.sort_values('pvalue', ascending=True)
results

,feature,pvalue,is_chosen
11,Credit Score,2.495861e-211,✅
23,Current Loan Amount-Monthly Debt_diff,6.899395e-88,✅
30,Monthly Debt-Current Loan Amount_diff,6.899395e-88,✅
8,Current Loan Amount,6.975268e-88,✅
26,Current Credit Balance-Current Loan Amount_diff,9.428961e-88,✅
22,Current Loan Amount-Current Credit Balance_diff,9.428961e-88,✅
20,Current Loan Amount-Annual Income_diff,3.220400e-86,✅
13,Annual Income-Current Loan Amount_diff,3.220400e-86,✅
21,Current Loan Amount-Maximum Open Credit_diff,9.622588e-66,✅
17,Maximum Open Credit-Current Loan Amount_diff,9.622588e-66,✅


Видим, что в десятку лучших попали не все признаки с p-value < 0.05. Также не понятно, сколько нужно признаков отбирать. На этот вопрос ответим в третьей части.

## 2. Отбор признаков с помощью метода RFE

Сущность метода: модель обучается на всех признаках и  оценивается их значимость. Затем исключается один или несколько наименее значимых признаков.  Модель снова обучается на оставшихся признаках, снова их оценивает и так далее, пока не останется заданное количество лучших признаков.

Отберем 10 признаков. Для предсказания используем модель LGBMClassifier. Укажем только, что данные несбалансированные (`is_unbalance=True`).

In [10]:
model = lgbm.LGBMClassifier(is_unbalance=True, random_state=42)

rfe = RFE(estimator=model, 
          n_features_to_select=10)

X_selected_RFE = rfe.fit_transform(X, y) # массив значений отобранных признаков
selected_features_RFE = X.columns[rfe.support_] # список отобранных признаков
# готовим таблицу с результатами
is_chosen = np.where(rfe.support_, '✅', '❌')
results = pd.DataFrame({'feature': num_features, 'rank': rfe.ranking_, 'is_chosen': is_chosen})
results = results.sort_values('rank', ascending=True)
results

,feature,rank,is_chosen
12,Annual Income-Maximum Open Credit_diff,1,✅
22,Current Loan Amount-Current Credit Balance_diff,1,✅
3,Years of Credit History,1,✅
18,Maximum Open Credit-Current Credit Balance_diff,1,✅
17,Maximum Open Credit-Current Loan Amount_diff,1,✅
8,Current Loan Amount,1,✅
9,Current Credit Balance,1,✅
10,Monthly Debt,1,✅
11,Credit Score,1,✅
13,Annual Income-Current Loan Amount_diff,1,✅


У отобранных признаков ранг равен 1. Чем хуже признак для модели, тем больше его ранг.

## 3. Сравнение методов

Обучим модель lightGBM на двух массивах данных и посчитаем f1-score на [кросс-валидации](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.cv.html) в функции `lgbm_cv()`.

In [11]:
# функция для расчета f1
def f1(y_pred, train_data):
    y_true = train_data.get_label()
    y_pred = np.where(y_pred < 0.5, 0, 1)  
    return 'f1', f1_score(y_true, y_pred), True

In [12]:
def lgbm_cv(data, labels):
    d_train = lgbm.Dataset(data, label=labels)
    params = {
        'objective' : 'binary',
        'is_unbalance': 'true',
    }
    cv_results = lgbm.cv(params, d_train, num_boost_round=100, nfold=3, 
                    verbose_eval=False, early_stopping_rounds=20, feval=f1,
                    seed=42)
    return data.shape[1], cv_results['f1-mean'][-1]

Если мы обучаем модель на всех числовых признаках:

In [13]:
n, score = lgbm_cv(X, y)
print('All {} features - best f1-score: {:.5f}'.format(n, score))

All 32 features - best f1-score: 0.52135


Если мы обучаем модель на признаках, отобранных моделью SelectKBest:

In [14]:
n, score = lgbm_cv(X[selected_features_kBest], y)
print('SelectKBest {} features - best f1-score: {:.5f}'.format(n, score))

SelectKBest 10 features - best f1-score: 0.50797


Если мы обучаем модель на признаках, отобранных моделью RFE:

In [15]:
n, score = lgbm_cv(X[selected_features_RFE], y)
print('RFE {} features - best f1-score: {:.5f}'.format(n, score))

RFE 10 features - best f1-score: 0.50679


10 отобранных признаков в обоих случаях показали худший результат, чем все признаки. Попробуем подобрать количество признаков.

Подбор количества признаков для модели SelectKBest:

In [16]:
scores = []
for k in range(10, X.shape[1] + 1, 1):
    fs = SelectKBest(score_func=f_classif, k=k) 
    X_selected = fs.fit_transform(X, y)
    n, score = lgbm_cv(X_selected, y)
    scores.append(score)
print('SelectKBest {} features - best f1-score: {:.5f}'.format(10 + np.argmax(scores), max(scores)))

SelectKBest 31 features - best f1-score: 0.52135


Подбор количества признаков для модели RFE:

In [17]:
scores = []
for k in range(10, X.shape[1] + 1, 1):
    rfe = RFE(estimator=model, 
          n_features_to_select=k)
    X_selected = rfe.fit_transform(X, y)
    n, score = lgbm_cv(X_selected, y)
    scores.append(score)
print('RFE {} features - best f1-score: {:.5f}'.format(10 + np.argmax(scores), max(scores)))

RFE 13 features - best f1-score: 0.53115


**Вывод:** Модель RFE показала себя лучше модели SelectKBest. С RFE метрика выше при меньшем количестве отобранных признаков. Тем более, что использование SelectKBest не совсем корректно без проверки признаков на нормальность.

Можно было бы еще эти способы сравнить с отбором на основе U-критерия Манна-Уитни (непараметрический критерий, используемый для сравнения средних двух генеральных совокупностей).